---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-machine-learning/resources/bANLa) course resource._

---

## Assignment 4 - Understanding and Predicting Property Maintenance Fines

This assignment is based on a data challenge from the Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)). 

The Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)) and the Michigan Student Symposium for Interdisciplinary Statistical Sciences ([MSSISS](https://sites.lsa.umich.edu/mssiss/)) have partnered with the City of Detroit to help solve one of the most pressing problems facing Detroit - blight. [Blight violations](http://www.detroitmi.gov/How-Do-I/Report/Blight-Complaint-FAQs) are issued by the city to individuals who allow their properties to remain in a deteriorated condition. Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance?

The first step in answering this question is understanding when and why a resident might fail to comply with a blight ticket. This is where predictive modeling comes in. For this assignment, your task is to predict whether a given blight ticket will be paid on time.

All data for this assignment has been provided to us through the [Detroit Open Data Portal](https://data.detroitmi.gov/). **Only the data already included in your Coursera directory can be used for training the model for this assignment.** Nonetheless, we encourage you to look into data from other Detroit datasets to help inform feature creation and model selection. We recommend taking a look at the following related datasets:

* [Building Permits](https://data.detroitmi.gov/Property-Parcels/Building-Permits/xw2a-a7tf)
* [Trades Permits](https://data.detroitmi.gov/Property-Parcels/Trades-Permits/635b-dsgv)
* [Improve Detroit: Submitted Issues](https://data.detroitmi.gov/Government/Improve-Detroit-Submitted-Issues/fwz3-w3yn)
* [DPD: Citizen Complaints](https://data.detroitmi.gov/Public-Safety/DPD-Citizen-Complaints-2016/kahe-efs3)
* [Parcel Map](https://data.detroitmi.gov/Property-Parcels/Parcel-Map/fxkw-udwf)

___

We provide you with two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.

Note: All tickets where the violators were found not responsible are not considered during evaluation. They are included in the training set as an additional source of data for visualization, and to enable unsupervised and semi-supervised approaches. However, they are not included in the test set.

<br>

**File descriptions** (Use only this data for training your model!)

    train.csv - the training set (all tickets issued 2004-2011)
    test.csv - the test set (all tickets issued 2012-2016)
    addresses.csv & latlons.csv - mapping from ticket id to addresses, and from addresses to lat/lon coordinates. 
     Note: misspelled addresses may be incorrectly geolocated.

<br>

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant


___

## Evaluation

Your predictions will be given as the probability that the corresponding blight ticket will be paid on time.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC). 

Your grade will be based on the AUC score computed for your classifier. A model which with an AUROC of 0.7 passes this assignment, over 0.75 will recieve full points.
___

For this assignment, create a function that trains a model to predict blight ticket compliance in Detroit using `train.csv`. Using this model, return a series of length 61001 with the data being the probability that each corresponding ticket from `test.csv` will be paid, and the index being the ticket_id.

Example:

    ticket_id
       284932    0.531842
       285362    0.401958
       285361    0.105928
       285338    0.018572
                 ...
       376499    0.208567
       376500    0.818759
       369851    0.018528
       Name: compliance, dtype: float32
       
### Hints

* Make sure your code is working before submitting it to the autograder.

* Print out your result to see whether there is anything weird (e.g., all probabilities are the same).

* Generally the total runtime should be less than 10 mins. You should NOT use Neural Network related classifiers (e.g., MLPClassifier) in this question. 

* Try to avoid global variables. If you have other functions besides blight_model, you should move those functions inside the scope of blight_model.

* Refer to the pinned threads in Week 4's discussion forum when there is something you could not figure it out.

In [59]:
import pandas as pd
import numpy as np
def blight_model():
    from sklearn.model_selection import train_test_split
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.model_selection import GridSearchCV
    from sklearn.metrics import confusion_matrix, classification_report, auc   
    from sklearn.linear_model import LogisticRegression
    import datetime as dt
    #############################
    df3 = pd.read_csv('addresses.csv')
    df4 = pd.read_csv(filepath_or_buffer='latlons.csv')
    df3.replace(to_replace='8325 joy rd, Detroit MI 482O4', value='8325 joy rd, Detroit MI', inplace=True)
    df3.replace(to_replace='1201 elijah mccoy dr, Detroit MI 48208', value='1201 elijah mccoy dr, Detroit MI', inplace=True)
    df3.replace(to_replace='12038 prairie, Detroit MI 482O4', value='12038 prairie, Detroit MI', inplace=True)
    df3.replace(to_replace='6200 16th st, Detroit MI 482O8', value='6200 16th st, Detroit MI', inplace=True)
    df_map = pd.merge(df3, df4, left_on = 'address', right_on = 'address', how='inner')
    # Your code here
    df = pd.read_csv('train.csv',  encoding='latin-1')
    df = df[['compliance','ticket_id', 'disposition', 'country',
             'judgment_amount', 'discount_amount', 'ticket_issued_date', 'state']]
    df  = df[df['country'] == 'USA']
    df['state']  = df['state'].apply(lambda x: 1 if x=='MI' else 0)
    df = pd.merge(df, df_map, left_on = 'ticket_id', right_on = 'ticket_id')
    df.dropna(inplace=True)
    df['ticket_issued_date'] = df['ticket_issued_date'].apply(lambda x: dt.datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))
    df['Year'] = df['ticket_issued_date'].apply(lambda x: x.year)
    df['Month'] = df['ticket_issued_date'].apply(lambda x: x.month)
    df['Day'] = df['ticket_issued_date'].apply(lambda x: x.day)
    df['Hour'] = df['ticket_issued_date'].apply(lambda x: x.hour)
    #df['ticket_issued_date'] = df['ticket_issued_date'].apply(lambda x: x.timestamp()) 
    df = pd.get_dummies(df, columns=['disposition'], drop_first=True)
    df.drop(['ticket_issued_date','ticket_id','address', 'country'], axis=1, inplace=True)
    df.dropna(inplace=True)
    X = df.iloc[:,1:]
    y = df.iloc[:,0]
    ############
    df2 = pd.read_csv('test.csv',  encoding='latin-1')
    df2 = df2[['ticket_id', 'disposition', 'judgment_amount', 'discount_amount', 'ticket_issued_date', 'state']]
    df2['state']  = df2['state'].apply(lambda x: 1 if x=='MI' else 0)
    df2 = pd.merge(df2, df_map, left_on = 'ticket_id', right_on = 'ticket_id')
    df2['ticket_issued_date'] = df2['ticket_issued_date'].apply(lambda x: dt.datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))
    df2['Year'] = df2['ticket_issued_date'].apply(lambda x: x.year)
    df2['Month'] = df2['ticket_issued_date'].apply(lambda x: x.month)
    df2['Day'] = df2['ticket_issued_date'].apply(lambda x: x.day)
    df2['Hour'] = df2['ticket_issued_date'].apply(lambda x: x.hour)
    #df2['ticket_issued_date'] = df2['ticket_issued_date'].apply(lambda x: x.timestamp())
    df2 = pd.get_dummies(df2, columns=['disposition'], drop_first=True)
    df2.drop(['ticket_issued_date','address', 'disposition_Responsible (Fine Waived) by Deter',
           'disposition_Responsible - Compl/Adj by Default',
           'disposition_Responsible - Compl/Adj by Determi',
            'disposition_Responsible by Dismissal'], axis=1, inplace=True)
    df2.fillna(method='ffill', inplace=True)
    ############
    logi = LogisticRegression(C=0.1, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=20, penalty='l1', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)
    logi.fit(X, y)
    y_prob = logi.predict_proba(df2.iloc[:,1:])
    df2['compliance'] = y_prob[:,1]
    df2.set_index('ticket_id', inplace=True)
    return df2['compliance']
blight_model()

/opt/conda/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2821: DtypeWarning: Columns (11,12,31) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


ticket_id
284932    0.409834
285362    0.256730
285361    0.386015
285338    0.377406
285346    0.393158
285345    0.377399
285347    0.401125
285342    0.874540
285530    0.299978
284989    0.390417
285344    0.392997
285343    0.255984
285340    0.262511
285341    0.401134
285349    0.429968
285348    0.413739
284991    0.390415
285532    0.346859
285406    0.347514
285001    0.331600
285006    0.268977
285405    0.286738
285337    0.406889
285496    0.425518
285497    0.401316
285378    0.293362
285589    0.366847
285585    0.369907
285501    0.425668
285581    0.262879
            ...   
376367    0.214186
376366    0.275453
376362    0.817135
376363    0.822054
376365    0.214186
376364    0.275453
376228    0.295642
376265    0.289308
376286    0.688212
376320    0.288597
376314    0.281367
376327    0.748974
376385    0.729241
376435    0.787022
376370    0.828592
376434    0.331242
376459    0.338387
376478    0.019239
376473    0.294719
376484    0.302073
376482    0.306395
37

In [ ]:
'''from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix, classification_report, auc   
import datetime as dt

df3 = pd.read_csv('addresses.csv')
df4 = pd.read_csv('latlons.csv')
df_map = pd.merge(df3, df4, left_on = 'address', right_on = 'address')
df_map.head()

df = pd.read_csv('train.csv',  encoding='latin-1')
df = df[['ticket_id', 'disposition', 'compliance', 'state', 'country',
         'judgment_amount', 'discount_amount', 'ticket_issued_date']]
df  = df[df['country'] == 'USA']
df['state']  = df['state'].apply(lambda x: 1 if x=='MI' else 0)
df = pd.merge(df, df_map, left_on = 'ticket_id', right_on = 'ticket_id')
df.dropna(inplace=True)
df['ticket_issued_date'] = df['ticket_issued_date'].apply(lambda x: dt.datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))
df['Year'] = df['ticket_issued_date'].apply(lambda x: x.year)
df['Month'] = df['ticket_issued_date'].apply(lambda x: x.month)
df['Day'] = df['ticket_issued_date'].apply(lambda x: x.day)
df['Hour'] = df['ticket_issued_date'].apply(lambda x: x.hour)
df['ticket_issued_date'] = df['ticket_issued_date'].apply(lambda x: x.timestamp()) 
df = pd.get_dummies(df, columns=['disposition'], drop_first=True)
df.drop(['ticket_issued_date','ticket_id','address', 'country'], axis=1, inplace=True)
df.dropna(inplace=True)
X = df.iloc[:,1:]
y = df.iloc[:,0]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=101)

df2 = pd.read_csv('test.csv',  encoding='latin-1')
df2 = df2[['ticket_id', 'disposition', 'judgment_amount', 'discount_amount', 'ticket_issued_date', 'state']]
df2['state']  = df2['state'].apply(lambda x: 1 if x=='MI' else 0)
df2 = pd.merge(df2, df_map, left_on = 'ticket_id', right_on = 'ticket_id')
df2.dropna(inplace=True)
df2['ticket_issued_date'] = df2['ticket_issued_date'].apply(lambda x: dt.datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))
df2['Year'] = df2['ticket_issued_date'].apply(lambda x: x.year)
df2['Month'] = df2['ticket_issued_date'].apply(lambda x: x.month)
df2['Day'] = df2['ticket_issued_date'].apply(lambda x: x.day)
df2['Hour'] = df2['ticket_issued_date'].apply(lambda x: x.hour)
df2['ticket_issued_date'] = df2['ticket_issued_date'].apply(lambda x: x.timestamp())
df2 = pd.get_dummies(df2, columns=['disposition'], drop_first=True)
df2.drop(['ticket_issued_date','ticket_id','address', 'disposition_Responsible (Fine Waived) by Deter',
       'disposition_Responsible - Compl/Adj by Default',
       'disposition_Responsible - Compl/Adj by Determi',
        'disposition_Responsible by Dismissal'], axis=1, inplace=True)
df2.dropna(inplace=True)

forest = RandomForestClassifier(n_jobs=20)
param = {'n_estimators':[100,200], 'class_weight':[{0:1, 1:10}, 'balanced']}
grid = GridSearchCV(forest, param_grid=param, verbose=2, cv=3, scoring='roc_auc')
grid.fit(X_train, y_train)
y_pred = grid.predict(X_test)      
print(classification_report(y_test, y_pred))

%matplotlib inline
from sklearn.metrics import roc_auc_score, roc_curve
print(roc_auc_score(y_test, y_pred))
x,y,_ = roc_curve(y_test, y_pred, pos_label=1)
import matplotlib.pyplot as plt
plt.plot(x,y)
plt.xlabel('False Positive Rate')
plt.ylabel('True Posiive Rate')

from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
logi = LogisticRegression(n_jobs=20)
param2 = {'C':[0.1,1,10], 'penalty':['l1','l2'], 'class_weight':[{0:1, 1:10},'balanced']}
grid2 = GridSearchCV(logi, param_grid=param2, verbose=2, cv=3, scoring='roc_auc')
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
grid2.fit(X_train, y_train)
y_pred2 = grid2.predict(X_test)      
print(classification_report(y_test, y_pred2))

%matplotlib inline
from sklearn.metrics import roc_auc_score, roc_curve
print(roc_auc_score(y_test, y_pred2))
x,y,_ = roc_curve(y_test, y_pred2, pos_label=1)
import matplotlib.pyplot as plt
plt.plot(x,y)
plt.xlabel('False Positive Rate')
plt.ylabel('True Posiive Rate')

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
plt.figure(figsize=[12,8])
sns.heatmap(df.corr(), annot=True)
plt.figure(figsize=[12,8])
sns.heatmap(df2.corr(), annot=True)

df2 = pd.read_csv('test.csv',  encoding='latin-1')
for index in range(len(df2.columns)):
    print (df2.columns[index], df2[df2.columns[index]].nunique())
    
df1 = pd.read_csv('train.csv',  encoding='latin-1')
for index in range(len(df1.columns)):
    print (df1.columns[index], df1[df1.columns[index]].nunique())
'''